# Running Large Language Models (LLMs) locally for Retrieval-Augmented-Generation (RAG) Systems with full privacy

**tl;dr:** You can run small LLMs locally on your consumer PC and with ollama that's very easy to set up. It is fun to chat with an LLM locally, but it gets really interesting when you build RAG systems or agents with your local LLM. I show you an example of a RAG-System built with llama-index.

## Running small LLMs locally with quantization

Large language models are large, mindboggingly large. Even if we had the source code and the weights of ChatGPTs GPT-4o model, with its (probably, the exact size is not known) 1,800b parameters - that is b for billion - it would be about 3 TB in size if every paramter is stored as a 16 bit float. Difficult to fit into your RAM!

`<rant>`
We could use proper SI notation, '1800G' or '1.8T' instead of '1800b' 😞, since 'billion' means different things in different languages, but here we are.
`</rant>`

But nevermind, we don't have the code and weights anyway. So what about open source models? While the flagships are still too large, there is a vibrant community on the HuggingFace platform that makes and improves models that have only **8b** to **30b** parameters, and those models are not useless. Meta has recently released a language model llama-3.2 with only **3b** parameters. While you cannot expect the same detailed knowledge about the world and attention span as the flagship models, these models still produce coherent text and you can have decent short conversations with them. I would recommend to use at least an **8b** model, because the smaller models likely won't follow your prompt very well.

An 8b model is 200 times smaller than GPT-4o, but still has a size of about 15 GB. It fits into your CPU RAM, but you want it to fit onto your GPU. If it does not fit completely onto the GPU, a part of the calculation has to be done with the CPU, and that will slow down the generation dramatically. Memory transfer speed is the bottleneck.

Fortunately, one can quantize the parameters quite strongly without loosing much. It turns out one can go down to 4 or 5 bits per parameter without loosing much (about one percent in benchmarks compared to the original model). This finally brings these models down to a size that fits onto consumer GPUs. You need some extra memory for the code and context window as well.

**If you are interested in this sort of thing and plan to buy a GPU soon, take one with at least 16 GB of RAM. GPU speed does not really matter.**

There are a couple of libraries which allow you to run these quantized models, but the best one by far is **Ollama** in my experience. Ollama is really easy to install and use. It successfully hides a lot of the complexity from you, and gives you easy start into the world of runnig local LLMs.

I had a lot of fun trying out different models. There are leaderboards ([Open LLM Leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard) and [Chatbot Arena](https://lmarena.ai/)) which help to select good candidates. I noticed large differences in perceived quality among models with the same size. Generally, I recommend finetuned versions of the llama-3.1:8b and gemma2:9b models by the community. If you want to skip over that, then try out mannix/gemma2-9b-simpo, and if you have at least 16GB of GPU RAM, gemma2:27b.

## Great, I have a local LLM running, now what?

Having an LLM running locally is nice and all, but for programming and asking questions about the world, the free tiers of ChatGPT and Claude are better. The real interesting use case for local LLMs is to chat with your documents using retrieval augmented generation (RAG).

There is great synergy in running a RAG System with a local LLM.
- You can keep your local documents private. Nothing will ever be transferred to the cloud.
- No additional costs. If you want to use the API of ChatGPT or Claude, you have to pay eventually. That's especially annoying while you are still developing, when you will run the LLMs over and over to test your application.
- Local LLMs lack detailed world knowledge, but the RAG-System complements that lack of knowledge. Without RAG, local LLMs hallucinate a lot, but with RAG they will provide factual knowledge.

A general advantage of RAG is that you can look into the text pieces that the LLM used to formulate its answer, which turns the LLM from a black box into a (nearly) white box.

## Building a simple RAG System with llama-index

For a RAG system, you need to convert your documents into plain text or Markdown, and an index to pull up relevant pieces from this corpus according to your query. There is currently gold-rush around developing converters for all kinds of documents into LLM-readable text, especially when it comes to PDFs. People try to make you to pay for this service. For PDFs, a free alternative that runs locally is **pymupdf4llm**. If your documents contain images, you can also run a multi-model LLM like llama-3.2-vision to make text descriptions for these images automatically.

Once you have your documents in plain text, you can split into mouth-sized pieces (mouth-sized for your LLM and its (small) context window) and use an embedding model to compute semantic vectors for each piece. These vectors magically encode semantic meaning of text, and can be used to find pieces that are relevant to a query using cosine similiarity - that's essentially a dot-product of the vectors. It is hard to believe that this works, but it actually does. Search via embeddings is superior to keyword search, but I can also say from experience that it is not a silver bullet. The best RAG Systems combine keywords with embeddings in some way. Using a good embedding model is key. If you use a model trained for english on German text, for example, it won't perform well, or if your documents contain lots of technical language that the embedding model was not trained on.

Thankfully, Ollama also offers embedding models, so you can run these locally as well. I found that mxbai-embed-large works well for both english and German text.

Writing a RAG from scratch with Ollama is not too hard, but it usually pays off to use a well-designed library to do the grunt work, and then start to improve from there. I compared many libraries, and can confidently recommend **llama-index** as the best one by far. It is feature-rich and well designed: little boilerplate code for simple things, yet easy to extend. The workflow system especially is really well designed. Just their (good) documentation is annoyingly difficult to find, they try to push you to their paid cloud services (did I mention, there is a gold rush...).

Below, I show you a RAG demo system, where I pull in Wikipedia pages about the seven antique world wonders, I then ask some questions about the Rhodes statue and the Hanging Gardens. As I am German, I wanted to see how well this works with German queries on German documents. That is not trivial, because both the LLM and the embedding model then have to understand German. I compare the result with and with RAG. Without RAG, the model will hallucinate details. With RAG, it follows the facts in the source documents closely. It is really impressive.

To run this, you need to install a couple of Python packages:

- ollama
- llama-index
- llama-index-llms-ollama
- llama-index-embeddings-ollama
- llama-index-readers-wikipedia
- wikipedia
- mistune
- ipython

Mistune renders Markdown to HTML.

In [3]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, VectorStoreIndex
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core.node_parser import SentenceSplitter
import textwrap
import mistune
from IPython.display import display_html


def wrap(s):
    return "\n".join(textwrap.wrap(s, replace_whitespace=False))


# logging.basicConfig(stream=sys.stdout, level=logging.INFO)

Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

Settings.llm = Ollama(model="mannix/gemma2-9b-simpo", request_timeout=1000)

Settings.text_splitter = SentenceSplitter(chunk_size=256, chunk_overlap=128)

In [4]:
# Load data from the German Wikipedia
documents = WikipediaReader().load_data(
    pages=[
        "Zeus-Statue des Phidias",
        "Tempel der Artemis in Ephesos",
        "Pyramiden von Gizeh",
        "Pharos von Alexandria",
        "Mausoleum von Halikarnassos",
        "Koloss von Rhodos",
        "Hängende Gärten der Semiramis",
    ], lang_prefix="de"
)

In [ ]:
# Lots of stuff is happening here. This splits the seven pages into chunks of texts, 
# and computes an embedding vector for each chunk, in the end we have 76 chunks of text
# that the LLM can use. We don't need to pass the embedding model or the text splitter
# explicitly, they are pulled from the Settings object.
index = VectorStoreIndex.from_documents(documents, show_progress=True)

c:\Users\HansDembinski\source\blog\.pixi\envs\default\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 212/212 [00:12<00:00, 16.39it/s]


In [10]:
# Some questions for the LLM about facts regarding two of the seven wonders
question = (
    "Aus welchen Materialien wurde der Koloss von Rhodos konstruiert?",
    "Beschreibe die Pose des Koloss von Rhodos.",
    "War der Koloss von Rhodos als nackte oder bekleidete Figur dargestellt?",
    "In welcher Stadt befanden sich die Hängenden Gärten?"
)

In [12]:
# A lot of stuff is happening here behind the scenes: a query engine is constructed
# from the document index. The query engine computes an embedding for the query, and
# selects 10 text pieces that are most similar to the query. It then prompts the 
# LLM with our question and provides the text pieces as context.
engine = index.as_query_engine(similarity_top_k=10)

show_sources = False

# Now we ask our questions. Set show_sources=True to see which text pieces were used.
# For reference, we compare the RAG answer ("RAG") with a plain LLM query ("Ohne RAG"). 
for q in question:
    q2 = q + " Antworte detailliert auf Deutsch."
    
    response = Settings.llm.complete(q2)
    rag = engine.query(q2)

    s = f"# {q}\n\n## Ohne RAG\n\n{wrap(response.text)}\n\n## RAG\n\n{wrap(rag.response)}"

    if show_sources:
        s += "\n\n## Sources\n\n"
        for node in rag.source_nodes:
            s += f"### Score {node.score}\n{wrap(node.text)}\n\n"

    s = mistune.html(s)
    display_html(s, raw=True)


Aus welchen Materialien wurde der Koloss von Rhodos konstruiert? 
 Ohne RAG 
 Obwohl der genaue Materialzusammensetzung des Kolosses von Rhodos
nicht vollständig überliefert ist, können wir aufgrund historischer
Beschreibungen und vergleichbarer antiker Bauwerke detailliert
rekonstruieren, aus welchen Materialien er wahrscheinlich errichtet
wurde: 
 1. Hauptstruktur und Sockel: 
 
 Kalkstein: Dies war
das dominierende Material für den Sockel und die Grundstruktur des
Kolosses. Rhodos verfügte über lokale Kalkstein-Abbaugebiete, welche
für massive Bauprojekte ideal waren. Kalkstein bietet Stabilität,
Widerstandsfähigkeit gegen Witterung und ermöglichte die präzise
Bearbeitung großer Blöcke. 
 Mörtel: Um die Kalksteinblöcke zu
verbinden, wurde ein Mörtel aus Zement, Sand und Kalk verwendet.
Dieser garantierte die dauerhafte Verbindung und Tragfähigkeit der
Konstruktion. 
 
 2. Skulpturale Elemente und Dekoration: 
 
 Bronze: Der Koloss selbst, insbesondere die lebensgroße Figur,
war höchstwahrscheinlich aus Bronze gefertigt. Bronze war damals das
Material der Wahl für monumentale Skulpturen aufgrund seiner
Haltbarkeit, Formbarkeit und glänzenden Oberfläche. Die Bronze wurde
vermutlich durch Schmieden und Gießen in großen Teilen hergestellt,
welche dann zusammengefügt wurden. 
 Vergoldung: Historische
Quellen erwähnen, dass der Koloss möglicherweise mit einer goldenen
Schicht bedeckt war, die seine Ausstrahlung und Pracht verstärkte.
Diese Vergoldung wäre auf Bronze-Basis aufgebracht worden und diente
als dekorativer und symbolischer Akzent. 
 Marmor und andere
Steine: Detaillierte Verzierungen, Gesichter, Kleidung und
Accessoires könnten aus Marmor, Alabaster oder anderen kostbaren
Steinen gefertigt worden sein, um dem Koloss eine feinere und
komplexere Optik zu verleihen. 
 
 3. Innenkonstruktion (geringfügig
bekannt): 
 
 Holz und Fachwerk: Wahrscheinlich nutzten die
Baumeister ein System aus Holz und Fachwerk innerhalb des Kolosses, um
die schwere Bronze-Struktur zu stabilisieren und Tragfähigkeit zu
gewährleisten. Dieses innere Gerüst wäre weniger sichtbar gewesen und
diente der Funktionalität. 
 
 Zusammenfassend lässt sich sagen, dass
der Koloss von Rhodos eine beeindruckende Kombination aus robusten
Steinen wie Kalkstein, der formbareren Bronze für die Skulptur und
dekorativen Elementen aus Gold, Marmor und anderen wertvollen
Materialien darstellte. Diese Materialwahl spiegelt die technische
Expertise und künstlerische Ambition der antiken Welt wider. 
 RAG 
 Der Koloss von Rhodos bestand aus Bronze und Eisen. Für die Stabilität
wurden im Inneren der Figur Eisengerüste und Steine eingesetzt.

Beschreibe die Pose des Koloss von Rhodos. 
 Ohne RAG 
 Der Koloss von Rhodos, obwohl nicht vollständig erhalten, lässt sich
anhand von Beschreibungen aus antiken Quellen und rekonstruktiven
Darstellungen in seiner Pose detailliert beschreiben:
 Grundhaltung: Der Koloss stand in erhobener, majestätischer
Haltung , symbolisch für die Stärke und Überlegenheit der Insel
Rhodos. Er war nicht sitzend oder kniend , sondern stehend auf
zwei Beinen . Seine Körperachse war leicht nach vorne geneigt ,
was Dynamik und Bewegung ins Bild brachte, obwohl er insgesamt
statisch wirkte. 
 Arme und Hände: Die Arme waren
 ausgebreitet , wobei die rechte Hand vermutlich in einer hoch
erhobenen Geste dargestellt wurde, möglicherweise einen
Friedenskranz oder eine ähnliche Symbolik haltend. Die linke Hand lag
 möglicherweise etwas tiefer, eventuell auf einem Schwert oder einem
anderen attributen , um Macht und Schutz zu demonstrieren. Die
genaue Position der Hände ist jedoch aufgrund fehlender Überreste
diskutabel. 
 Kopf und Blick: Der Kopf war hoch erhoben , der
Blick vermutlich frontal gerichtet , symbolisch auf das Meer und
die Reisenden gerichtet, die auf Rhodos ankamen. Dies unterstrich die
Rolle des Kolosses als Wachmann und Garant der Sicherheit. Sein
Gesichtsausdruck wurde wahrscheinlich als seren und weise 
interpretiert, unterstreichend die Weisheit und Stärke der Stadt
Rhodos. 
 Gesamteindruck: Die Pose des Kolosses war eine
 monumentale Darstellung von Macht, Schutz und friedvoller
Überlegenheit . Sie kombinierte Eleganz, Würde und eine leicht
dynamische Geste, um eine beeindruckende Figur zu erschaffen, die den
Besucher von Rhodos willkommen hieß und gleichzeitig seine Stärke
symbolisierte. 
 Obwohl der Koloss heute nur noch in Fragmenten
existiert, lässt sich seine beeindruckende Pose durch die historische
Überlieferung und kunstgeschichtliche Analyse noch immer deutlich
rekonstruieren und bewundern. 
 RAG 
 Obwohl keine direkte antike Darstellung existiert, wird der Koloss in
neueren Rekonstruktionen meist als ruhig stehende Figur dargestellt,
mit der Annahme eines gestreckten, stabilen Standbilds. Hervorhebung
der Pose durch einen spreizbeinigen Stand oder dynamische Bewegung
wird in den Textquellen nicht erwähnt.

War der Koloss von Rhodos als nackte oder bekleidete Figur dargestellt? 
 Ohne RAG 
 Der Koloss von Rhodos, eines der sieben Weltwunder der Antike, wurde
 mit Kleidung dargestellt, obwohl Details darüber, welche
spezifischen Kleidungsstücke er trug, unsicher sind. 
 Hier die
Erkenntnisse und Schlussfolgerungen, die auf historische Hinweise und
vergleichbare Darstellungen anderer antiker Statuen basieren: 
 
 Keine eindeutigen bildlichen Aufzeichnungen: Es existieren keine
direkte, detaillierte Abbildungen des Kolosses, die seine Kleidung
zweifelsfrei zeigen. 
 Traditionelle Darstellung von
Herrscherstatuen: In der griechischen Kunst der damaligen Zeit,
besonders bei kolossalen Statuen von Göttern oder Herrschern, war es
üblich, sie in kleidetem Zustand zu repräsentieren. Dies
symbolisierte Macht, Würde und Ansehen. Der Koloss soll den
Sonnengott Helios darstellen, und ähnliche Gottheiten wurden oft mit
Gewändern oder Roben abgebildet. 
 Vergleich mit anderen Werken: 
Statuen ähnlicher Größe und Funktion aus derselben Epoche, wie
beispielsweise der Zeus-Statue in Olympia, waren ebenfalls bekleidet.
Dies deutet auf einen vergleichbaren Stil für den Koloss hin. 
 Interpretationen und künstlerische Freiheit: Obwohl konkrete
Kleidungsdetails unbekannt sind, ließen sich drapedierende Gewänder,
eine Mantelform oder sogar militärische Elemente (Rhodos war bekannt
für seine Seefahrer und Kriegerkultur) als plausible
Darstellungsweisen des Kolosses interpretieren. Künstler hätten
basierend auf der Funktion und dem Symbolismus der Statue
künstlerische Freiheit genossen. 
 
 Zusammenfassend lässt sich sagen,
dass der wissenschaftliche Konsens darauf hindeutet, der Koloss von
Rhodos wurde als bekleidete Figur dargestellt, obwohl die genaue Art
der Kleidung aufgrund fehlender direkter Quellen rekonstruiert werden
muss. 
 RAG 
 Obwohl keine direkte, antike Darstellung des Kolosses existiert, lässt
sich aufgrund von Schlussfolgerungen und Parallelen vermuten, dass er
als nackter junger Mann dargestellt wurde. Der Text erwähnt ein
Vorbild in Münzbildern von Rhodos, die aus dem späten 5. Jahrhundert
v. Chr. stammen und meist nackte männliche Figuren zeigen, sowie die
gängige Vorstellung von Helios als Gott, oft in der klassischen Kunst
nackt abgebildet. Dadurch wird eine bekleidete Darstellung
unwahrscheinlicher.

In welcher Stadt befanden sich die Hängenden Gärten? 
 Ohne RAG 
 Die Hängenden Gärten, eines der Sieben Weltwunder der Antike, befanden
sich in der Stadt Semiramis (auch bekannt als Babylon ),
gelegen im historischen Mesopotamien, dem heutigen Irak . 
 Obwohl
ihre genaue Lage innerhalb Babels bis heute nicht endgültig geklärt
ist, vermutet man, dass sie sich in der Nähe des Palastes
Nebukadnezars II. befanden, dem babylonischen König, der oft mit der
Errichtung der Gärten in Verbindung gebracht wird. Archäologische
Funde und historische Quellen deuten auf eine Position am
Euphratufer hin, wo Wasser für die Bewässerung der beeindruckenden
Terrassenanlagen leicht verfügbar gewesen wäre. 
 Es ist wichtig zu
erwähnen, dass die Existenz und genaue Beschreibung der Hängenden
Gärten trotz zahlreicher literarischer Erwähnungen bis heute
Gegenstand historischer Debatten sind. Während archäologische Beweise
für einen komplexen Gartenbau in Babylon existieren, fehlt ein
eindeutiger Beweis für die spezifische Architektur und Dimension der
"h hängenden" Gärten, wie sie in antiken Texten beschrieben werden.
Dennoch bleibt die Vorstellung von diesen üppigen, terrassierten
Gärten, die aus der trockenen Landschaft Babels emporragten, ein
ikonisches Symbol für menschliche Ingenieurskunst und den Wunsch nach
Schönheit inmitten des Ödlands. 
 RAG 
 Der Text legt nahe, dass die Hängenden Gärten in unmittelbarer Nähe
eines Palastes und innerhalb eines Areals mit dem Namen "Südburg"
lagen. Weitere Hinweise deuten auf eine Verbindung zu einem
"Nordpalast" und einem Außenwerk hin. Zusammenfassend lässt sich
sagen, dass die Hängenden Gärten innerhalb einer Stadt vermutet
werden, die diese architektonischen Strukturen und Bezeichnungen
aufwies.

The answers without the RAG are much nicer to read, but contain halluciations, while the RAG answers are dull, brief, but factually correct. The behavior of the LLM without RAG is a consequence of human preference optimization. The LLM generates answers by default that *look nice* to humans.

The RAG answer is very short, because the internal prompt of llama-index asks the LLM to only use information provided by the RAG system for its answer and not use its internal knowledge. It is therefore not a bug but a feature that the answer of the LLM is so short: it tries to only make statements that are covered by the text pieces. 

1. Question is about the materials used to construct the Rhodes statue.

The standard LLM claims that wood was used in the construction of the Rhodes statue, but there are no records in the Wikipedia at least about that. The RAG answer is factual correct, it mentions the four materials that are described in the Wikipedia article.

2. Question is about the pose of the Rhodes statue.

The standard LLM gives a lot of hallucinated detail. We don't know much about the pose, and the short RAG answer summarises that.

3. Question is about whether the statue was clothed or naked.

The standard LLM says it was clothed, probably because a lot of the antique statues were clothed, but the RAG answer is correct, the statue was naked as far as we know.

4. Question is about the city in which the Hanging Gardens were supposed to be located.

The standard LLM gives the correct answer in this case, Babylon. The RAG answer speaks about the location relative to the palace, but does not mention the city. This is the only case where the RAG answer is worse, although not factually incorrect. The failure in this case is related to the index, which fails to retrieve the right text piece with the information.